In [ ]:
import tensorflow as tf
import numpy as np
import math
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Activation,BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler,EarlyStopping
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import SGD,Adam

In [ ]:
df_train,df_test = tfds.load('kmnist',split=['train','test'],shuffle_files=True,batch_size=-1)

Shuffling and writing examples to /root/tensorflow_datasets/kmnist/3.0.1.incompleteG6ITRK/kmnist-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/kmnist/3.0.1.incompleteG6ITRK/kmnist-test.tfrecord


Dataset kmnist downloaded and prepared to /root/tensorflow_datasets/kmnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
def step_decay(epoch):
	initial_lrate = 0.1
	drop = 0.5
	epochs_drop = 10.0
	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lrate

In [ ]:
x_train,y_train = df_train['image']/255,df_train['label']
x_test,y_test = df_test['image']/255,df_test['label']

In [ ]:
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std)
x_test = (x_test-mean)/(std)

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
y_train.shape

(60000, 10)

In [ ]:
y_test.shape

(10000, 10)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
    )
datagen.fit(x_train)

In [ ]:
weight_decay = 1e-5
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [ ]:
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

In [ ]:
model.compile(optimizer=SGD(0.0),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(datagen.flow(x_train,y_train,batch_size=32),validation_data=(x_test,y_test),epochs=40,shuffle=True,callbacks=callbacks_list)

Epoch 1/40
1875/1875 [==============================] - 22s 11ms/step - loss: 1.4903 - accuracy: 0.7448 - val_loss: 0.2738 - val_accuracy: 0.9201
Epoch 2/40
1875/1875 [==============================] - 21s 11ms/step - loss: 0.2011 - accuracy: 0.9405 - val_loss: 0.1895 - val_accuracy: 0.9512
Epoch 3/40
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1417 - accuracy: 0.9573 - val_loss: 0.1989 - val_accuracy: 0.9504
Epoch 4/40
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1133 - accuracy: 0.9675 - val_loss: 0.1592 - val_accuracy: 0.9615
Epoch 5/40
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0901 - accuracy: 0.9734 - val_loss: 0.1368 - val_accuracy: 0.9682
Epoch 6/40
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0881 - accuracy: 0.9750 - val_loss: 0.1173 - val_accuracy: 0.9718
Epoch 7/40
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0717 - accuracy: 0.9796 - val_loss: 0.1536 -

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()